## Tutorial: Merging Datasets, Training, and Validation for YOLO-based Object Detection

Welcome to the comprehensive tutorial on merging multiple YOLO-formatted datasets, training a YOLO-NAS model, and validating its performance using mAP evaluation. This guide is structured to help you understand the data structure, configure your YAML files, and utilize the provided Python scripts effectively.

# 1. Understanding the Data Structure

Before diving into merging and training, it’s essential to comprehend how your data is organized.

### Data Organization

Each dataset follows a structured directory format with separate folders for training, validation, and testing images and their corresponding labels. Here’s how the data is organized:
```
/path/to/dataset/
├── train/
│   ├── images/
│   │   ├── image_1.jpg
│   │   ├── image_2.jpg
│   │   └── ...
│   └── labels/
│       ├── image_1.txt
│       ├── image_2.txt
│       └── ...
├── val/
│   ├── images/
│   │   ├── image_3.jpg
│   │   ├── image_4.jpg
│   │   └── ...
│   └── labels/
│       ├── image_3.txt
│       ├── image_4.txt
│       └── ...
├── test/
│   ├── images/
│   │   ├── image_5.jpg
│   │   ├── image_6.jpg
│   │   └── ...
│   └── labels/
│       ├── image_5.txt
│       ├── image_6.txt
│       └── ...
└── annotations/
    ├── data.yaml
    └── yolo_format_train/
    └── yolo_format_val/
    └── yolo_format_test/
```

## Or little change but still each part has its own folder and unique ids

Key Points:
	•	Unique Image Names: Each image must have a unique name across all datasets. This uniqueness is crucial for the merging process. Fortunately, platforms like Roboflow ensure that all images have unique names by default.
	•	One-to-One Relationship: There is a strict one-to-one relationship between images and label files. For every image (.jpg, .jpeg, or .png), there must be a corresponding label file (.txt) with the same base name.
	•	YOLO 5 Label Format: Label files follow the YOLO 5 format:

```
class_id center_x center_y width height
```



## 2. Configuration Files

Configuration files are pivotal in defining how datasets are merged and how the training process is orchestrated. We’ll discuss two primary YAML files: config.yaml and data.yaml.

### a. config.yaml

This file specifies the datasets you want to merge and the parameters for the merged dataset.

Sample config.yaml:

```
# config.yaml

# List of datasets to merge
datasets:
  - name: "coco_yolo"
    data_dir: "/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo"
    train_images_dir: "train/"
    train_labels_dir: "annotations/yolo_format_train"
    val_images_dir: "val/"
    val_labels_dir: "annotations/yolo_format_val"
    test_images_dir: "test/"
    test_labels_dir: "annotations/yolo_format_test"
  
  - name: "merged_forklift"
    data_dir: "/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/merged_forklift"
    train_images_dir: "train/"
    train_labels_dir: "annotations/yolo_format_train"
    val_images_dir: "val/"
    val_labels_dir: "annotations/yolo_format_val"
    test_images_dir: "test/"
    test_labels_dir: "annotations/yolo_format_test"

# Parameters for the merged dataset
merged_dataset:
  data_dir: "/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/merged_coco_forklift"
  train_images_dir: "train/images"
  train_labels_dir: "train/labels"
  val_images_dir: "valid/images"
  val_labels_dir: "valid/labels"
  test_images_dir: "test/images"
  test_labels_dir: "test/labels"

```

Explanation of config.yaml Components:
•	datasets: A list of datasets you intend to merge.

•	name: Identifier for the dataset.

•	data_dir: Root directory of the dataset.

•	train_images_dir: Directory containing training images relative to data_dir.

•	train_labels_dir: Directory containing training labels relative to data_dir.

•	val_images_dir: Directory containing validation images relative to data_dir.

•	val_labels_dir: Directory containing validation labels relative to data_dir.

•	test_images_dir: Directory containing testing images relative to data_dir.

•	test_labels_dir: Directory containing testing labels relative to data_dir.

•	merged_dataset: Parameters defining where and how the merged dataset will be organized.

•	data_dir: Root directory for the merged dataset.

•	train_images_dir: Directory for merged training images relative to data_dir.

•	train_labels_dir: Directory for merged training labels relative to data_dir.

•	val_images_dir: Directory for merged validation images relative to data_dir.

•	val_labels_dir: Directory for merged validation labels relative to data_dir.

•	test_images_dir: Directory for merged testing images relative to data_dir.

•	test_labels_dir: Directory for merged testing labels relative to data_dir.


Important Notes:

•	Class Mapping: During the merge, classes with similar names differing only in case (e.g., “Person” vs. “person”) are considered identical by converting them to lowercase.


### b. data.yaml

After merging, a new data.yaml file is generated, which includes unified class names and dataset paths.

Sample data.yaml:

```
names:
  - person
  - forklift
  - goggles
  - helmet
  - no-goggles
  - no-helmet
  - no-vest
  - vest
nc: 8
test: /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/merged_dataset/test
train: /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/merged_dataset/train
val: /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/merged_dataset/val
```

Explanation of data.yaml Components:
•	names: List of class names in the order they appear. The order is crucial as it determines the class indices used during training and evaluation.

•	nc: Number of classes.

•	test: Path to the testing images directory.

•	train: Path to the training images directory.

•	val: Path to the validation images directory.

Class Mapping (class_mapping.yaml):

During the merge process, a class_mapping.yaml file is also created to map original class indices to the unified indices.

## 3. Merging Datasets

The merging process consolidates multiple datasets into a single unified dataset, ensuring consistent class mappings and organized directories.

### Running the Merge Script

Use the merge_datasets_script.py to merge your datasets based on the config.yaml configuration.

Command:
```
python merge_datasets_script.py --config config.yaml
```
Script Flags and Examples

•	--config: (Required) Path to your config.yaml file.

What Happens During the Merge:
1.	Parsing Configuration: The script reads the config.yaml to identify datasets to merge and the parameters for the merged dataset.

2.	Merging data.yaml Files: It consolidates the data.yaml files from each dataset, unifying class names and assigning new class indices. Classes with the same name (case-insensitive) are merged into a single class.

3.	Creating class_mapping.yaml: Generates a class_mapping.yaml that maps original class indices from each dataset to the unified class indices.

4.	Combining Datasets: Copies images and labels from each dataset into the merged directories (train/images, val/images, test/images, etc.), updating label files to reflect the new class indices.

5.	Analyzing Merged Dataset: Performs integrity checks to ensure that all images have corresponding label files and vice versa.

#### Sample Output:

After running the merge script, you should see outputs indicating the progress, such as the number of classes merged, class mappings, and any discrepancies in the datasets.

Example Output:
```
--- Merging data.yaml Files ---

Merged Classes:
0: person
1: forklift
2: goggles
3: helmet
4: no-goggles
5: no-helmet
6: no-vest
7: vest

Class Mappings per Dataset:

Dataset: /home/user/projects/object_detection/data/coco_yolo/annotations/data.yaml
  Original Index 0 -> New Index 0
  Original Index 1 -> New Index 1
  ...

Dataset: /home/user/projects/object_detection/data/merged_forklift/annotations/data.yaml
  Original Index 0 -> New Index 0
  Original Index 1 -> New Index 1
  ...

Class mappings have been saved to /home/user/projects/object_detection/data/merged_coco_forklift/class_mappings.yaml

--- Merging Datasets ---

...

--- Analyzing Merged Datasets ---

...

--- Verification Summary ---
  Train Split: OK (Merged: 1000, Source: 1000)
  Val Split: OK (Merged: 200, Source: 200)
  Test Split: OK (Merged: 300, Source: 300)

--- Analysis Completed ---
```


## 4. Training the Model

With the datasets merged, the next step is to train your YOLO-NAS model using the model_training.py script.

Running the Training Script

Command:
```
python model_training.py --data_yaml /path/to/merged_coco_forklift/data.yaml
```

Script Flags and Examples
•	--data_yaml: (Required) Path to your merged data.yaml file.

•	--checkpoint_dir: (Optional) Directory where checkpoints will be saved. If not specified, defaults to checkpoints.

•	--experiment_name: (Optional) Name for the training experiment. Useful for organizing multiple training runs.


Examples:

1.	Basic Training:
    ```
    python model_training.py --data_yaml /home/user/projects/object_detection/data/merged_coco_forklift/data.yaml
    ```
2.	Specify Checkpoint Directory:
    ```
    python model_training.py \
    --data_yaml /home/user/projects/object_detection/data/merged_coco_forklift/data.yaml \
    --checkpoint_dir /home/user/projects/object_detection/checkpoints
    ```
3.	Custom Experiment Name:
    ```
    python model_training.py \
    --data_yaml /home/user/projects/object_detection/data/merged_coco_forklift/data.yaml \
    --experiment_name "custom_yolonas_run"
    ```
4. Combined Example:
    ```
    python model_training.py \
    --data_yaml /home/user/projects/object_detection/data/merged_coco_forklift/data.yaml \
    --checkpoint_dir /home/user/projects/object_detection/checkpoints \
    --experiment_name "custom_yolonas_run"
    ```

What Happens During Training:
1.	Parsing data.yaml: Extracts dataset paths and class names, ensuring the order is preserved.

2.	Setting Up Data Loaders: Configures training and validation data loaders with specified batch sizes and number of workers.

3.	Initializing the Trainer and Model:
	
    •	Trainer: Manages the training process, saving checkpoints, and logging.
    
    •	Model: Initializes the YOLO-NAS model with pre-trained weights from COCO and sets the number of classes based on data.yaml.

4.	Configuring Training Parameters: Sets hyperparameters such as learning rate schedules, optimizer settings, loss functions, and metrics.

5.	Starting Training: Begins the training process, periodically saving checkpoints and evaluating performance on the validation set.

6.	Checkpoint Management: YOLO-NAS automatically creates subfolders within the checkpoint directory, named with the date and time of the training run (e.g., 2024-04-27_15-30-00), and saves the best model as ckpt_best.pth.

Important Notes:

•	Adjusting Training Settings: If you wish to modify training settings (e.g., batch size, learning rate), adjust them directly in the model_training.py script under the train_params dictionary.

•	Automatic Checkpointing: The script automatically searches for the latest checkpoint based on the subfolder naming convention.

## 5. Validating the Model

After training, validating the model’s performance using mAP (mean Average Precision) is crucial. The validation_script.py handles this process.

#### Running the Validation Script

Command:
```
python validation_script.py \
    --data_yaml /path/to/merged_coco_forklift/data.yaml \
    --checkpoints_base_dir /path/to/checkpoints_base_dir \
    --mAP_output_move_dir /path/to/save/mAP/results
```
Script Flags and Examples
	
•	--data_yaml: (Required) Path to your merged data.yaml file.
	
•	--checkpoints_base_dir: (Required) Base directory containing checkpoint subdirectories.
	
•	--mAP_output_move_dir: (Optional) Directory to move mAP evaluation results. This is essential because the default mAP output directory will be overwritten in future runs.
	
•	--mAP_script_path: (Optional) Path to the mAP evaluation script. Defaults to /home/LIBS/jozefov/projects/object_detection_avaltar/libs/mAP/main.py.

Examples:

1.	Basic Validation:
```
python validation_script.py \
    --data_yaml /home/user/projects/object_detection/data/merged_coco_forklift/data.yaml \
    --checkpoints_base_dir /home/user/projects/object_detection/checkpoints_base_dir
```

2.	Specify mAP Output Move Directory:
```
python validation_script.py \
    --data_yaml /home/user/projects/object_detection/data/merged_coco_forklift/data.yaml \
    --checkpoints_base_dir /home/user/projects/object_detection/checkpoints_base_dir \
    --mAP_output_move_dir /home/user/projects/object_detection/mAP_results
```

3.	Custom mAP Script Path:
```
python validation_script.py \
    --data_yaml /home/user/projects/object_detection/data/merged_coco_forklift/data.yaml \
    --checkpoints_base_dir /home/user/projects/object_detection/checkpoints_base_dir \
    --mAP_script_path /home/user/projects/object_detection/libs/mAP/custom_main.py
```

4.	Combined Example:
```
python validation_script.py \
    --data_yaml /home/user/projects/object_detection/data/merged_coco_forklift/data.yaml \
    --checkpoints_base_dir /home/user/projects/object_detection/checkpoints_base_dir \
    --mAP_output_move_dir /home/user/projects/object_detection/mAP_results \
    --mAP_script_path /home/user/projects/object_detection/libs/mAP/custom_main.py
```

Validation Process Overview:

1.	Parsing data.yaml: Extracts dataset paths and class names, ensuring the order is preserved.

2.	Finding the Latest Checkpoint: Automatically searches for the latest checkpoint (ckpt_best.pth) within the specified checkpoints directory. YOLO-NAS organizes checkpoints into subdirectories named with the date and time, facilitating easy identification of the latest model.

3.	Loading the Best Model:
    
    •	Loads the YOLO-NAS model using the latest checkpoint.
	
    •	Moves the model to the appropriate device (GPU if available, else CPU).

4.	Preparing Validation Images:
	
    •	Lists all .jpg images in the validation directory.
	
    •	Sorts the image paths to maintain consistent ordering for predictions.

5.	Performing Predictions:
	
    •	Runs the model to predict bounding boxes on validation images with a confidence threshold of 0.20.

6.	Cleaning mAP Input Directories:
	
    •	Removes existing contents in the mAP input directories to prevent interference with new evaluations.
	
    •	Recreates the necessary directories (ground-truth, detection-results, images-optional).

7.	Generating Ground Truth and Detection Results:
	
    •	Utilizes the generate_ground_truth function to prepare the ground truth and detection results required for mAP evaluation.
	
    •	Ensures that class names are mapped correctly and in the order defined in data.yaml.

8.	Running mAP Evaluation:

    •	Executes the mAP evaluation script to compute performance metrics.

9.	Moving mAP Output (Optional):
	
    •	Moves the evaluation results to a specified directory to prevent them from being overwritten in future runs.

Script Flags and Examples

•	--data_yaml: (Required) Path to your merged data.yaml file.

•	--checkpoints_base_dir: (Required) Base directory containing checkpoint subdirectories.

•	--mAP_output_move_dir: (Optional) Directory to move mAP evaluation results.

•	--mAP_script_path: (Optional) Path to the mAP evaluation script. Defaults to /home/LIBS/jozefov/projects/object_detection_avaltar/libs/mAP/main.py.

Example Commands:
1.	Basic Validation:
```
python validation_script.py \
    --data_yaml /home/user/projects/object_detection/data/merged_coco_forklift/data.yaml \
    --checkpoints_base_dir /home/user/projects/object_detection/checkpoints_base_dir
```

2. Validation with mAP Output Directory:
```
python validation_script.py \
    --data_yaml /home/user/projects/object_detection/data/merged_coco_forklift/data.yaml \
    --checkpoints_base_dir /home/user/projects/object_detection/checkpoints_base_dir \
    --mAP_output_move_dir /home/user/projects/object_detection/mAP_results
```

3.	Validation with Custom mAP Script Path:
```
python validation_script.py \
    --data_yaml /home/user/projects/object_detection/data/merged_coco_forklift/data.yaml \
    --checkpoints_base_dir /home/user/projects/object_detection/checkpoints_base_dir \
    --mAP_script_path /home/user/projects/object_detection/libs/mAP/custom_main.py
```

## 6. Additional Notes

#### Automated Mapping and Class Indexing
	
•	Automatic Class Mapping: The merging script automatically handles class mapping by converting class names to lowercase and merging classes that are identical in a case-insensitive manner. This ensures that classes like “Person” and “person” are treated as the same class, preventing duplication and ensuring consistency.
	
•	Order Preservation: The order of classes as they appear in data.yaml is preserved. This is crucial because YOLO-NAS relies on this order for class indexing during training and evaluation.

#### Checkpoint Management
	
•	Automatic Checkpoint Searching: During validation, the script automatically searches for the latest checkpoint by sorting the subdirectories (which are named with date and time) and selecting the most recent one (ckpt_best.pth).
	
•	Checkpoint Directory Structure: YOLO-NAS organizes checkpoints into subdirectories named with the date and time of the training run (e.g., 2024-04-27_15-30-00). This structure helps in easily identifying and retrieving the latest or specific checkpoints.

#### mAP Evaluation Output Management
	
•	Cleaning Before Evaluation: Before performing a new mAP evaluation, the script cleans the existing mAP input directories to ensure that previous results do not interfere with the current evaluation.
	
•	Moving Evaluation Results: Since mAP evaluation results are overwritten in subsequent runs, it’s recommended to move the results to a separate directory after each evaluation. Use the --mAP_output_move_dir flag to specify the destination directory.
	
•	Directory Creation: If the specified destination directory for mAP results does not exist, the script automatically creates it, preventing errors during the move operation.


### Running Scripts in Order

1.	Merging Datasets:
```
python merge_datasets_script.py --config config.yaml
```

2.	Training the Model:
```
python model_training.py --data_yaml /path/to/merged_coco_forklift/data.yaml
```

**With Additional Flags:**
```
python model_training.py \
    --data_yaml /path/to/data.yaml \
    --checkpoint_dir /path/to/checkpoints \
    --experiment_name "custom_run"
```

3.	Validating the Model:
```
python validation_script.py \
    --data_yaml /path/to/merged_coco_forklift/data.yaml \
    --checkpoints_base_dir /path/to/checkpoints_base_dir \
    --mAP_output_move_dir /path/to/save/mAP/results
```